In [3]:
conda install pytorch torchvision -c pytorch

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 22.11.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /root/anaconda3/envs/tf1

  added / updated specs:
    - pytorch
    - torchvision


The following packages will be UPDATED:

  ca-certificates    intel::ca-certificates-2022.07.19-h06~ --> pkgs/main::ca-certificates-2022.10.11-h06a4308_0 None
  certifi            intel::certifi-2022.6.15-py37h06a4308~ --> pkgs/main::certifi-2022.9.24-py37h06a4308_0 None


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [4]:
conda install -c intel scikit-learn

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 22.11.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /root/anaconda3/envs/tf1

  added / updated specs:
    - scikit-learn


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates-2022.10.11~ --> intel::ca-certificates-2022.07.19-h06a4308_0 None
  certifi            pkgs/main::certifi-2022.9.24-py37h06a~ --> intel::certifi-2022.6.15-py37h06a4308_0 None


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [45]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim

import sklearn

In [19]:
data = pd.read_csv("ugib-2.csv")
data

hadm_id  subject_id   stay_id               intime  \
0     21080282    14134823  33993411  2181-09-06 18:03:00   
1     24970864    16342642  36182548  2200-11-10 19:38:00   
2     27196888    13408013  31942840  2159-03-11 18:50:00   
3     28432198    16154757  39675268  2116-12-02 00:57:00   
4     21765388    10640857  37541138  2180-04-25 17:45:00   
...        ...         ...       ...                  ...   
3286  26927108    19871603  34703633  2175-04-28 01:00:00   
3287  25399885    13492618  34544661  2151-04-13 21:43:00   
3288  22194922    17741944  33794308  2180-10-15 14:03:00   
3289  22978484    13105864  35870981  2110-12-26 05:05:00   
3290  23376128    16417021  37321080  2165-07-03 19:31:00   

                  outtime gender                    race        age  \
0     2181-09-07 01:09:00      F                   WHITE  50.681784   
1     2200-11-10 23:57:00      F                   WHITE  66.859548   
2     2159-03-11 22:20:00      M                   WHITE  55.191310   
3     2116-12-02 02:49:00      F                   WHITE  34.918965   
4     2180-04-25 21:10:00      M                   WHITE  52.317101   
...                   ...    ...                     ...        ...   
3286  2175-04-28 02:59:00      F                   WHITE  85.320623   
3287  2151-04-14 05:01:00      F  BLACK/AFRICAN AMERICAN  34.281772   
3288  2180-10-15 21:19:00      M                   WHITE  25.790832   
3289  2110-12-26 10:53:00      M                   WHITE  67.983994   
3290  2165-07-03 22:06:00      F                   WHITE  69.503374   

               trans_time  trans  
0     2181-09-07 01:24:00      1  
1     2200-11-11 00:42:00      1  
2     2159-03-11 22:24:00      1  
3     2116-12-02 04:00:00      1  
4     2180-04-27 17:15:00      1  
...                   ...    ...  
3286                  NaN      0  
3287                  NaN      0  
3288                  NaN      0  
3289                  NaN      0  
3290                  NaN      0  

[3291 rows x 10 columns]

In [105]:
dataframe = pd.DataFrame({"id": data.hadm_id,
    "gender": data.gender,
    "race": data.race,
    "age": data.age,                      
    "label": data.trans
})
dataframe.head(10)

id gender                    race        age  label
0  21080282      F                   WHITE  50.681784      1
1  24970864      F                   WHITE  66.859548      1
2  27196888      M                   WHITE  55.191310      1
3  28432198      F                   WHITE  34.918965      1
4  21765388      M                   WHITE  52.317101      1
5  20478909      F                   WHITE  78.716528      1
6  28725627      M                   WHITE  92.221257      1
7  20614940      M                   WHITE  43.848912      1
8  28194053      M  BLACK/AFRICAN AMERICAN  80.886068      1
9  22130611      M                   WHITE  71.891958      1

In [98]:
dataframe.label.value_counts()


0    2521
1     770
Name: label, dtype: int64

In [102]:
# change the race and gender into values
dataframe['gender'] = dataframe['gender'].replace(['F'], 1)
dataframe['gender'] = dataframe['gender'].replace(['M'], 0)
dataframe['race'] = dataframe['race'].replace(['WHITE'], 1)
dataframe['race'] = dataframe['race'].replace(['BLACK/AFRICAN AMERICAN', 'HISPANIC/LATINO - DOMINICAN',
       'WHITE - OTHER EUROPEAN', 'OTHER', 'ASIAN','UNKNOWN'],0)
dataframe['race'] = dataframe['race'].replace(['ASIAN - CHINESE', 'BLACK/CARIBBEAN ISLAND', 'BLACK/CAPE VERDEAN',
       'ASIAN - SOUTH EAST ASIAN', 'HISPANIC/LATINO - PUERTO RICAN',
       'WHITE - RUSSIAN', 'WHITE - EASTERN EUROPEAN'],0)
dataframe['race'] = dataframe['race'].replace(['PATIENT DECLINED TO ANSWER', 'MULTIPLE RACE/ETHNICITY',
       'HISPANIC/LATINO - HONDURAN', 'UNABLE TO OBTAIN', 'PORTUGUESE',
       'HISPANIC/LATINO - COLUMBIAN', 'HISPANIC OR LATINO'],0)
dataframe['race'] = dataframe['race'].replace(['AMERICAN INDIAN/ALASKA NATIVE', 'BLACK/AFRICAN',
       'WHITE - BRAZILIAN', 'HISPANIC/LATINO - GUATEMALAN',
       'HISPANIC/LATINO - SALVADORAN', 'HISPANIC/LATINO - MEXICAN',
       'SOUTH AMERICAN'],0)
dataframe['race'] = dataframe['race'].replace(['NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER',
       'ASIAN - ASIAN INDIAN', 'ASIAN - KOREAN'],0)
dataframe['gender']=dataframe['gender'].astype(float)
dataframe['race']=dataframe['race'].astype(float)

In [103]:
dataframe.head(10)

id  gender  race        age  label
0  21080282     1.0   1.0  50.681784      1
1  24970864     1.0   1.0  66.859548      1
2  27196888     0.0   1.0  55.191310      1
3  28432198     1.0   1.0  34.918965      1
4  21765388     0.0   1.0  52.317101      1
5  20478909     1.0   1.0  78.716528      1
6  28725627     0.0   1.0  92.221257      1
7  20614940     0.0   1.0  43.848912      1
8  28194053     0.0   0.0  80.886068      1
9  22130611     0.0   1.0  71.891958      1

In [104]:
features = list(dataframe)
target = ['label']

In [94]:
race = dataframe.race
gender = dataframe.gender
age = dataframe.age
label = dataframe.label


In [97]:
raceT = torch.tensor(race)
genderT = torch.tensor(gender)
ageT = torch.tensor(age)
labelT = torch.tensor(label)


In [ ]:
# split data into train and test
from sklearn.model_selection import train_test_split

race_train, gender_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, # 20% test, 80% train
                                                    random_state=42) # make the random split reproducible

len(X_train), len(X_test), len(y_train), len(y_test)